In [1]:
# @author: katelynsmith

import numpy as np # for maths 
import matplotlib # for plotting 
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
import matplotlib.pyplot as plt
from matplotlib import cm, ticker
#plt.style.use(['no-latex']) 
#from emcee import MASER
import glob
import os 
import pandas as pd

import astropy.units as u
from astropy.timeseries import BoxLeastSquares

import scipy.signal as sps

import maser as ms
import starspot as ss
import astrobase as ab
#from astrobase import periodbase


from tqdm import tqdm 
# Setting directory

ddir = '/Users/katelynsmith/Desktop/Capstone/'

import warnings
warnings.filterwarnings("ignore")

import latex

/Users/katelynsmith/opt/anaconda3/envs/example/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Defining variables 

M_s = 0.3 # Star mass (solar masses) #### REFERENCE https://exoplanetarchive.ipac.caltech.edu/overview/GJ%20625%20b#star_GJ-625_collapsible #####
R_s = 0.3 # Star radius (solar radii) ####  REFERENCE https://exoplanetarchive.ipac.caltech.edu/overview/GJ%20625%20b#star_GJ-625_collapsible  ####
P_s = 77.8 # Star rotation period (days) ####  REFERENCE https://exoplanetarchive.ipac.caltech.edu/overview/GJ%20625%20b#star_GJ-625_collapsible ####


i_s = 90 # Star inclination of the rotation axis relative to the line of sight (degrees) ####  parameter is being varied ####
B_s = 90.3 # Star dipole field strength at the magnetic poles (Gauss) #### http://www.diva-portal.org/smash/get/diva2:1600719/FULLTEXT01.pdf #### - total mag field strength known to be 30.1 G and this paper suggests the dipolar field strength is three times that value
beta = 20 # Star magnetic obliquity (degrees) #### parameter is being varied ####


phi_s0 = 0.2 # Star rotation phase at times = 0 (0 – 1) <<<<<< LEFT AS IS
i_p = 89.18 # Planet inclination of the orbital axis relative to the line of sight (degrees) #### REFERENCE (PAPER 7 on one note) #### - UNKNOWN FOR THIS PLANET SO KEPT THE SAME 
lam = 0 # Planet projected spin-orbit angle (degrees) #### ranging from -15 to 18 degrees -- LEFT AS IS
#REFERENCE Spin-orbit alignment and magnetic activity in the young planetary system AU Mic⋆ ####


P_p = 14.628 # GJ 625b period in days #### REFERENCE https://exoplanetarchive.ipac.caltech.edu/overview/GJ%20625%20b#planet_GJ-625-b_collapsible ####
a = 0.0178  # Planet orbital distance (stellar radii) #### REFERENCE https://exoplanets.nasa.gov/exoplanet-catalog/7184/gj-625-b/ ####
phi_p0 = 0.6 # Planet orbital phase at times = 0 (0 – 1) <<<<< LEFT AS IS

f = 10 # Emission observing frequency (MHz) #### - MAY INCREASE TO 3 GHz REFERENCE (PAPER 6 on one note) ####
alpha = 75 # Emission cone opening angle (degrees) <<<<< LEFT THESE AS IS - BASED ON KAVANAGH 2023
dalpha = 5 # Emission cone thickness (degrees) <<<<< LEFT THESE AS IS - BASED ON KAVANAGH 2023

In [6]:
def period(M_s, R_s, P_s, i_s, B_s, beta, phi_s0, a, i_p, lam, phi_p0, f, alpha, dalpha, times, interval):
   north_vis, south_vis = ms.maser(M_s, R_s, P_s, i_s, B_s, beta, phi_s0, a, i_p, lam, phi_p0, f, alpha, dalpha, times)

   visibility_North = north_vis*1.0
   visibility_South = south_vis*1.0

   if np.count_nonzero(visibility_North) > 0:
      rotate_N = ss.RotationModel(times, visibility_North, None)
      #rotate_N = ss.RotationModel(times, north_vis, None)
      #north_acf = ss.simple_acf(times, north_vis, interval)
      acf_period_N = rotate_N.acf_rotation(interval=interval)

      diffs_N = np.diff(visibility_North,1)
      ups_N = times[1:][diffs_N>0]
      downs_N = times[1:][diffs_N<0]
      
      if len(ups_N) > len(downs_N):
         ups_N_adj = [ups_N[i] for i in range(0, len(downs_N))]
         centres_N = (ups_N_adj+downs_N)/2.
         if len(centres_N)%2 != 0:
            centres_N = centres_N[:-1]
      
         durations_N = downs_N - ups_N_adj
         mean_duration_N = np.mean(durations_N)

         paired_peaks_centre_N = [(centres_N[i] + centres_N[i+1])/2 for i in range(0, len(centres_N)-1, 2)]
         
         if len(paired_peaks_centre_N)%2 == 0:
            pair_spacings_N = [(paired_peaks_centre_N[i+1] - paired_peaks_centre_N[i]) for i in range(0, len(paired_peaks_centre_N)-1, 2)]
            mean_pair_spacing_N = np.mean(pair_spacings_N)
            second_pair_spacings_N = [(paired_peaks_centre_N[i+2] - paired_peaks_centre_N[i]) for i in range(0, len(paired_peaks_centre_N)-2, 2)]
            mean_second_pair_spacing_N = np.mean(second_pair_spacings_N)
            third_pair_spacings_N = [(paired_peaks_centre_N[i+3] - paired_peaks_centre_N[i]) for i in range(0, len(paired_peaks_centre_N)-3, 2)]
            mean_third_pair_spacing_N = np.mean(third_pair_spacings_N)
         else:
            #mean_pair_spacing_N = np.nan
            paired_peaks_centre_N = paired_peaks_centre_N[:-1]
            pair_spacings_N = [(paired_peaks_centre_N[i+1] - paired_peaks_centre_N[i]) for i in range(0, len(paired_peaks_centre_N)-1, 2)]
            mean_pair_spacing_N = np.mean(pair_spacings_N)
            second_pair_spacings_N = [(paired_peaks_centre_N[i+2] - paired_peaks_centre_N[i]) for i in range(0, len(paired_peaks_centre_N)-2, 2)]
            mean_second_pair_spacing_N = np.mean(second_pair_spacings_N)
            third_pair_spacings_N = [(paired_peaks_centre_N[i+3] - paired_peaks_centre_N[i]) for i in range(0, len(paired_peaks_centre_N)-3, 2)]
            mean_third_pair_spacing_N = np.mean(third_pair_spacings_N)

         # spacings_N = np.diff(centres_N)
         # mean_spacing_N = np.mean(spacings_N)

         
         # second_spacings_N = [(centres_N[i+2] - centres_N[i]) for i in range(0, len(centres_N)-2, 1)]
         # mean_second_spacing_N = np.mean(second_spacings_N)

         
         # third_spacings_N = [(centres_N[i+3] - centres_N[i]) for i in range(0, len(centres_N)-3, 1)]
         # mean_third_spacing_N = np.mean(third_spacings_N)

      

      elif len(ups_N) < len(downs_N):
         downs_N_adj = [downs_N[i] for i in range(0, len(ups_N))]
         centres_N = (ups_N+downs_N_adj)/2.
         if len(centres_N)%2 != 0:
            centres_N = centres_N[:-1]
         
         durations_N = downs_N_adj - ups_N
         mean_duration_N = np.mean(durations_N)

         paired_peaks_centre_N = [(centres_N[i] + centres_N[i+1])/2 for i in range(0, len(centres_N)-1, 2)]
         
         if len(paired_peaks_centre_N)%2 == 0:
            pair_spacings_N = [(paired_peaks_centre_N[i+1] - paired_peaks_centre_N[i]) for i in range(0, len(paired_peaks_centre_N)-1, 2)]
            mean_pair_spacing_N = np.mean(pair_spacings_N)
            second_pair_spacings_N = [(paired_peaks_centre_N[i+2] - paired_peaks_centre_N[i]) for i in range(0, len(paired_peaks_centre_N)-2, 2)]
            mean_second_pair_spacing_N = np.mean(second_pair_spacings_N)
            third_pair_spacings_N = [(paired_peaks_centre_N[i+3] - paired_peaks_centre_N[i]) for i in range(0, len(paired_peaks_centre_N)-3, 2)]
            mean_third_pair_spacing_N = np.mean(third_pair_spacings_N)
         else:
            #mean_pair_spacing_N = np.nan
            paired_peaks_centre_N = paired_peaks_centre_N[:-1]
            pair_spacings_N = [(paired_peaks_centre_N[i+1] - paired_peaks_centre_N[i]) for i in range(0, len(paired_peaks_centre_N)-1, 2)]
            mean_pair_spacing_N = np.mean(pair_spacings_N)
            second_pair_spacings_N = [(paired_peaks_centre_N[i+2] - paired_peaks_centre_N[i]) for i in range(0, len(paired_peaks_centre_N)-2, 2)]
            mean_second_pair_spacing_N = np.mean(second_pair_spacings_N)
            third_pair_spacings_N = [(paired_peaks_centre_N[i+3] - paired_peaks_centre_N[i]) for i in range(0, len(paired_peaks_centre_N)-3, 2)]
            mean_third_pair_spacing_N = np.mean(third_pair_spacings_N)

         # spacings_N = np.diff(centres_N)
         # mean_spacing_N = np.mean(spacings_N)

         
         # second_spacings_N = [(centres_N[i+2] - centres_N[i]) for i in range(0, len(centres_N)-2, 1)]
         # mean_second_spacing_N = np.mean(second_spacings_N)

         
         # third_spacings_N = [(centres_N[i+3] - centres_N[i]) for i in range(0, len(centres_N)-3, 1)]
         # mean_third_spacing_N = np.mean(third_spacings_N)


      else:
         ups_N = ups_N
         downs_N = downs_N
         centres_N = (ups_N+downs_N)/2.
         if len(centres_N)%2 != 0:
            centres_N = centres_N[:-1]
         
         durations_N = downs_N - ups_N
         mean_duration_N = np.mean(durations_N)

         paired_peaks_centre_N = [(centres_N[i] + centres_N[i+1])/2 for i in range(0, len(centres_N)-1, 2)]
         
         if len(paired_peaks_centre_N)%2 == 0:
            pair_spacings_N = [(paired_peaks_centre_N[i+1] - paired_peaks_centre_N[i]) for i in range(0, len(paired_peaks_centre_N)-1, 2)]
            mean_pair_spacing_N = np.mean(pair_spacings_N)
            second_pair_spacings_N = [(paired_peaks_centre_N[i+2] - paired_peaks_centre_N[i]) for i in range(0, len(paired_peaks_centre_N)-2, 2)]
            mean_second_pair_spacing_N = np.mean(second_pair_spacings_N)
            third_pair_spacings_N = [(paired_peaks_centre_N[i+3] - paired_peaks_centre_N[i]) for i in range(0, len(paired_peaks_centre_N)-3, 2)]
            mean_third_pair_spacing_N = np.mean(third_pair_spacings_N)
         else:
            #mean_pair_spacing_N = np.nan
            paired_peaks_centre_N = paired_peaks_centre_N[:-1]
            pair_spacings_N = [(paired_peaks_centre_N[i+1] - paired_peaks_centre_N[i]) for i in range(0, len(paired_peaks_centre_N)-1, 2)]
            mean_pair_spacing_N = np.mean(pair_spacings_N)
            second_pair_spacings_N = [(paired_peaks_centre_N[i+2] - paired_peaks_centre_N[i]) for i in range(0, len(paired_peaks_centre_N)-2, 2)]
            mean_second_pair_spacing_N = np.mean(second_pair_spacings_N)
            third_pair_spacings_N = [(paired_peaks_centre_N[i+3] - paired_peaks_centre_N[i]) for i in range(0, len(paired_peaks_centre_N)-3, 2)]
            mean_third_pair_spacing_N = np.mean(third_pair_spacings_N)

         # spacings_N = np.diff(centres_N)
         # mean_spacing_N = np.mean(spacings_N)

         
         # second_spacings_N = [(centres_N[i+2] - centres_N[i]) for i in range(0, len(centres_N)-2, 1)]
         # mean_second_spacing_N = np.mean(second_spacings_N)

         
         # third_spacings_N = [(centres_N[i+3] - centres_N[i]) for i in range(0, len(centres_N)-3, 1)]
         # mean_third_spacing_N = np.mean(third_spacings_N)
   else:
      acf_period_N = np.nan
      mean_duration_N = np.nan
     # mean_spacing_N = np.nan
     # mean_second_spacing_N = np.nan
     # mean_third_spacing_N = np.nan
      mean_pair_spacing_N = np.nan
      mean_second_pair_spacing_N = np.nan 
      mean_third_pair_spacing_N = np.nan

   if np.count_nonzero(visibility_South) > 0:
      rotate_S = ss.RotationModel(times, visibility_South, None)
      #rotate_S = ss.RotationModel(times, south_vis, None)
      #south_acf = ss.simple_acf(times, south_vis, interval)
      acf_period_S = rotate_S.acf_rotation(interval=interval)

      diffs_S = np.diff(visibility_South,1)
      ups_S = times[1:][diffs_S>0]
      downs_S = times[1:][diffs_S<0]
      
      if len(ups_S) > len(downs_S):
         ups_S_adj = [ups_S[i] for i in range(0, len(downs_S))]
         centres_S = (ups_S_adj+downs_S)/2.
         if len(centres_S)%2 != 0:
            centres_S = centres_S[:-1]
      
         durations_S = downs_S - ups_S_adj
         mean_duration_S = np.mean(durations_S)

         paired_peaks_centre_S = [(centres_S[i] + centres_S[i+1])/2 for i in range(0, len(centres_S)-1, 2)]
         
         if len(paired_peaks_centre_S)%2 == 0:
            pair_spacings_S = [(paired_peaks_centre_S[i+1] - paired_peaks_centre_S[i]) for i in range(0, len(paired_peaks_centre_S)-1, 2)]
            mean_pair_spacing_S = np.mean(pair_spacings_S)
            second_pair_spacings_S = [(paired_peaks_centre_S[i+2] - paired_peaks_centre_S[i]) for i in range(0, len(paired_peaks_centre_S)-2, 2)]
            mean_second_pair_spacing_S = np.mean(second_pair_spacings_S)
            third_pair_spacings_S = [(paired_peaks_centre_S[i+3] - paired_peaks_centre_S[i]) for i in range(0, len(paired_peaks_centre_S)-3, 2)]
            mean_third_pair_spacing_S = np.mean(third_pair_spacings_S)
         else:
            #mean_pair_spacing_N = np.nan
            paired_peaks_centre_S = paired_peaks_centre_S[:-1]
            pair_spacings_S = [(paired_peaks_centre_S[i+1] - paired_peaks_centre_S[i]) for i in range(0, len(paired_peaks_centre_S)-1, 2)]
            mean_pair_spacing_S = np.mean(pair_spacings_S)
            second_pair_spacings_S = [(paired_peaks_centre_S[i+2] - paired_peaks_centre_S[i]) for i in range(0, len(paired_peaks_centre_S)-2, 2)]
            mean_second_pair_spacing_S = np.mean(second_pair_spacings_S)
            third_pair_spacings_S = [(paired_peaks_centre_S[i+3] - paired_peaks_centre_S[i]) for i in range(0, len(paired_peaks_centre_S)-3, 2)]
            mean_third_pair_spacing_S = np.mean(third_pair_spacings_S)

         # spacings_S = np.diff(centres_S)
         # mean_spacing_S = np.mean(spacings_S)

         
         # second_spacings_S = [(centres_S[i+2] - centres_S[i]) for i in range(0, len(centres_S)-2, 1)]
         # mean_second_spacing_S = np.mean(second_spacings_S)

         
         # third_spacings_S = [(centres_S[i+3] - centres_S[i]) for i in range(0, len(centres_S)-3, 1)]
         # mean_third_spacing_S = np.mean(third_spacings_S)
      
      elif len(ups_S) < len(downs_S):
         downs_S_adj = [downs_S[i] for i in range(0, len(ups_S))]
         centres_S = (ups_S+downs_S_adj)/2.
         if len(centres_S)%2 != 0:
            centres_S = centres_S[:-1]
         
         durations_S = downs_S_adj - ups_S
         mean_duration_S = np.mean(durations_S)

         paired_peaks_centre_S = [(centres_S[i] + centres_S[i+1])/2 for i in range(0, len(centres_S)-1, 2)]
         #pair_spacings_S = np.diff(paired_peaks_centre_S)
         if len(paired_peaks_centre_S)%2 == 0:
            pair_spacings_S = [(paired_peaks_centre_S[i+1] - paired_peaks_centre_S[i]) for i in range(0, len(paired_peaks_centre_S)-1, 2)]
            mean_pair_spacing_S = np.mean(pair_spacings_S)
            second_pair_spacings_S = [(paired_peaks_centre_S[i+2] - paired_peaks_centre_S[i]) for i in range(0, len(paired_peaks_centre_S)-2, 2)]
            mean_second_pair_spacing_S = np.mean(second_pair_spacings_S)
            third_pair_spacings_S = [(paired_peaks_centre_S[i+3] - paired_peaks_centre_S[i]) for i in range(0, len(paired_peaks_centre_S)-3, 2)]
            mean_third_pair_spacing_S = np.mean(third_pair_spacings_S)
         else:
            #mean_pair_spacing_N = np.nan
            paired_peaks_centre_S = paired_peaks_centre_S[:-1]
            pair_spacings_S = [(paired_peaks_centre_S[i+1] - paired_peaks_centre_S[i]) for i in range(0, len(paired_peaks_centre_S)-1, 2)]
            mean_pair_spacing_S = np.mean(pair_spacings_S)
            second_pair_spacings_S = [(paired_peaks_centre_S[i+2] - paired_peaks_centre_S[i]) for i in range(0, len(paired_peaks_centre_S)-2, 2)]
            mean_second_pair_spacing_S = np.mean(second_pair_spacings_S)
            third_pair_spacings_S = [(paired_peaks_centre_S[i+3] - paired_peaks_centre_S[i]) for i in range(0, len(paired_peaks_centre_S)-3, 2)]
            mean_third_pair_spacing_S = np.mean(third_pair_spacings_S)
         


         #spacings_S = np.diff(centres_S)
         #mean_spacing_S = np.mean(spacings_S)

         #second_spacings_S = [(centres_S[i+2] - centres_S[i]) for i in range(0, len(centres_S)-2, 1)]
         #mean_second_spacing_S = np.mean(second_spacings_S)

         #third_spacings_S = [(centres_S[i+3] - centres_S[i]) for i in range(0, len(centres_S)-3, 1)]
         #mean_third_spacing_S = np.mean(third_spacings_S)
      else:
         ups_S = ups_S
         downs_S = downs_S
         centres_S = (ups_S+downs_S)/2.
         if len(centres_S)%2 != 0:
            centres_S = centres_S[:-1]
         
         durations_S = downs_S - ups_S
         mean_duration_S = np.mean(durations_S)

         paired_peaks_centre_S = [(centres_S[i] + centres_S[i+1])/2 for i in range(0, len(centres_S)-1, 2)]
         #pair_spacings_S = np.diff(paired_peaks_centre_S)
         if len(paired_peaks_centre_S)%2 == 0:
            pair_spacings_S = [(paired_peaks_centre_S[i+1] - paired_peaks_centre_S[i]) for i in range(0, len(paired_peaks_centre_S)-1, 2)]
            mean_pair_spacing_S = np.mean(pair_spacings_S)
            second_pair_spacings_S = [(paired_peaks_centre_S[i+2] - paired_peaks_centre_S[i]) for i in range(0, len(paired_peaks_centre_S)-2, 2)]
            mean_second_pair_spacing_S = np.mean(second_pair_spacings_S)
            third_pair_spacings_S = [(paired_peaks_centre_S[i+3] - paired_peaks_centre_S[i]) for i in range(0, len(paired_peaks_centre_S)-3, 2)]
            mean_third_pair_spacing_S = np.mean(third_pair_spacings_S)
         else:
            #mean_pair_spacing_N = np.nan
            paired_peaks_centre_S = paired_peaks_centre_S[:-1]
            pair_spacings_S = [(paired_peaks_centre_S[i+1] - paired_peaks_centre_S[i]) for i in range(0, len(paired_peaks_centre_S)-1, 2)]
            mean_pair_spacing_S = np.mean(pair_spacings_S)
            second_pair_spacings_S = [(paired_peaks_centre_S[i+2] - paired_peaks_centre_S[i]) for i in range(0, len(paired_peaks_centre_S)-2, 2)]
            mean_second_pair_spacing_S = np.mean(second_pair_spacings_S)
            third_pair_spacings_S = [(paired_peaks_centre_S[i+3] - paired_peaks_centre_S[i]) for i in range(0, len(paired_peaks_centre_S)-3, 2)]
            mean_third_pair_spacing_S = np.mean(third_pair_spacings_S)

         #spacings_S = np.diff(centres_S)
         #mean_spacing_S = np.mean(spacings_S)

        
        # second_spacings_S = [(centres_S[i+2] - centres_S[i]) for i in range(0, len(centres_S)-2, 1)]
        # mean_second_spacing_S = np.mean(second_spacings_S)

       
        # third_spacings_S = [(centres_S[i+3] - centres_S[i]) for i in range(0, len(centres_S)-3, 1)]
        # mean_third_spacing_S = np.mean(third_spacings_S)
   else:
      acf_period_S = np.nan
      mean_duration_S = np.nan
     # mean_spacing_S = np.nan
     # mean_second_spacing_S = np.nan
     # mean_third_spacing_S = np.nan
      mean_pair_spacing_S = np.nan
      mean_second_pair_spacing_S = np.nan 
      mean_third_pair_spacing_S = np.nan

   #return acf_period_N, acf_period_S, mean_duration_N, mean_duration_S, mean_spacing_N, mean_spacing_S, mean_second_spacing_N, mean_second_spacing_S, mean_third_spacing_N, mean_third_spacing_S, mean_pair_spacing_N, mean_pair_spacing_S
   return acf_period_N, acf_period_S, mean_duration_N, mean_duration_S, mean_pair_spacing_N, mean_pair_spacing_S, mean_second_pair_spacing_N, mean_second_pair_spacing_S, mean_third_pair_spacing_N, mean_third_pair_spacing_S

In [9]:
times = np.linspace(0, 50, 20000) # increase to 100 days
interval = np.diff(times)[0]

betas = np.linspace(beta*0.5, beta*4, 100) # 50 and beta*2
inclinations = np.linspace(i_s*0.5, i_s*4, 100) # 50 and i_s*2


period_grid = np.zeros((len(betas), len(inclinations),2))
duration_grid =  np.zeros((len(betas), len(inclinations),2))
#first_spacing_grid = np.zeros((len(betas), len(inclinations),2))
#second_spacing_grid = np.zeros((len(betas), len(inclinations),2))
#third_spacing_grid = np.zeros((len(betas), len(inclinations),2))
pair_spacing_grid = np.zeros((len(betas), len(inclinations),2))
second_pair_spacing_grid = np.zeros((len(betas), len(inclinations),2))
third_pair_spacing_grid = np.zeros((len(betas), len(inclinations),2))

for j, b in (enumerate(tqdm(betas))):
    for k, inc in enumerate(inclinations):
        period_N, period_S, duration_N, duration_S, pair_spacing_N, pair_spacing_S, pair_spacing_2_N, pair_spacing_2_S, pair_spacing_3_N, pair_spacing_3_S = period(M_s, R_s, P_s, inc, B_s, b, phi_s0, a, i_p, lam, phi_p0, f, alpha, dalpha, times, interval)
        
        period_grid[j, k, :] = period_N, period_S
        duration_grid[j, k, :] = duration_N, duration_S
        #first_spacing_grid[j, k, :] = spacing_1_N, spacing_1_S
        #second_spacing_grid[j, k, :] = spacing_2_N, spacing_2_S
        #third_spacing_grid[j, k, :] = spacing_3_N, spacing_3_S
        pair_spacing_grid[j, k, :] = pair_spacing_N, pair_spacing_S
        second_pair_spacing_grid[j, k, :] = pair_spacing_2_N, pair_spacing_2_S
        third_pair_spacing_grid[j, k, :] = pair_spacing_3_N, pair_spacing_3_S

  0%|          | 0/100 [00:00<?, ?it/s]

 83%|████████▎ | 83/100 [1:37:45<20:01, 70.67s/it]


IndexError: arrays used as indices must be of integer (or boolean) type